In [1]:
import re
import spacy
from spacy.matcher import Matcher
import fitz
import os

In [2]:
# Descargar el modelo si no está instalado
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    from spacy.cli import download
    download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")
    

In [3]:
#def remove_extra_spaces(text):
#    return re.sub(r"\s+", " ", text.strip())

In [4]:
def extraer_metadatos_gaceta(ruta_pdf):
    """
    Extrae metadatos de una gaceta del Congreso usando un enfoque híbrido.
    """
    metadatos = {}

    try:
        with fitz.open(ruta_pdf) as doc:  
            texto_completo = ""
            for pagina in doc:
                texto_completo += pagina.get_text()
                
            
            texto_completo = preprocesar_texto(texto_completo)

            # 1. Nombre Principal
            if re.search(r"(?i)GACETA\s+DEL\s+CONGRESO", texto_completo):
                metadatos["nombre"] = "Gaceta del Congreso"

            # --- ISSN ---
            metadatos["issn"] = extraer_con_regex(r"I\s*S\s*S\s*N\s*(0\s*1\s*2\s*3\s*-\s*9\s*0\s*6\s*6)", texto_completo)
            if metadatos["issn"]:
                metadatos["issn"] = metadatos["issn"].replace(" ", "")

            # --- 2. Fecha ---
            patron_fecha = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
            coincidencia_fecha = re.search(patron_fecha, texto_completo)
            if coincidencia_fecha:
                meses = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
                         "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
                dia = int(coincidencia_fecha.group(2))
                mes = meses[coincidencia_fecha.group(3).lower()]
                anio = int(coincidencia_fecha.group(4))
                metadatos["fecha"] = f"{dia:02d}/{mes:02d}/{anio}"

            # 3. Entidad (Senado/Cámara)
            match_entity = re.search(r"(?i)(SENADO Y CÁMARA|SENADO|CÁMARA)", texto_completo)
            if match_entity:
                entity = match_entity.group(1).strip()
                if entity == "SENADO Y CÁMARA":
                    metadatos["entidades"] = ["SENADO Y CAMARA", "SENADO", "CÁMARA"]
                else:
                    metadatos["entidades"] = [entity]

            # --- 4. Año (Números Romanos) y Número de Publicación ---
            patron_anio_numero = r"AÑO\s+([MDCLXVI]+)\s*-\s*N[°º]\s*(\d+)"
            coincidencia_anio_numero = re.search(patron_anio_numero, texto_completo, re.IGNORECASE)
            if coincidencia_anio_numero:
                anio_romano = coincidencia_anio_numero.group(1).strip()
                numero_publicacion = coincidencia_anio_numero.group(2).strip()

                metadatos["anio_romano"] = anio_romano
                metadatos["anio"] = romano_a_entero(anio_romano)
                metadatos["numero_publicacion"] = int(numero_publicacion)

            # --- 5. Rama Legislativa ---
            pattern_branch = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+PÚBLICO"
            if re.search(pattern_branch, texto_completo):
                metadatos["rama"] = "Rama Legislativa del Poder Público"

            # --- 6. Directores (Enfoque Híbrido Robusto) ---
            metadatos["directores"] = {"senado": {}, "camara": {}}
            patron_directores_inicio = r"DIRECTORES:"
            coincidencia_inicio = re.search(patron_directores_inicio, texto_completo, re.IGNORECASE)

            if coincidencia_inicio:
                inicio_seccion = coincidencia_inicio.end()
                patron_directores_fin = r"(?:SECRETARIO GENERAL DEL SENADO|SECRETARIO GENERAL DE LA CÁMARA|RAMA LEGISLATIVA)"
                coincidencia_fin = re.search(patron_directores_fin, texto_completo[inicio_seccion:], re.IGNORECASE)

                if coincidencia_fin:
                    fin_seccion = inicio_seccion + coincidencia_fin.start()
                    texto_directores = texto_completo[inicio_seccion:fin_seccion]

                    lineas = texto_directores.splitlines()

                    nombre_senado = ""
                    cargo_senado = ""
                    nombre_camara = ""
                    cargo_camara = ""

                    for linea in lineas:
                        linea_upper = linea.upper()

                        if "SECRETARIO GENERAL DEL SENADO" in linea_upper:
                            cargo_senado = "SECRETARIO GENERAL DEL SENADO"
                            nombre_senado_match = re.search(r"([A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+(?:\s+[A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+)+)", linea)
                            if nombre_senado_match:
                                nombre_senado = nombre_senado_match.group(1).strip()
                            

                        elif "SECRETARIO GENERAL DE LA CÁMARA" in linea_upper:
                            cargo_camara = "SECRETARIO GENERAL DE LA CÁMARA"
                            nombre_camara_match = re.search(r"([A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+(?:\s+[A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+)+)", linea)
                            if nombre_camara_match:
                                nombre_camara = nombre_camara_match.group(1).strip()
                            

                    if nombre_senado and cargo_senado:
                        metadatos["directores"]["senado"]["nombre"] = nombre_senado
                        metadatos["directores"]["senado"]["cargo"] = cargo_senado
                    if nombre_camara and cargo_camara:
                        metadatos["directores"]["camara"]["nombre"] = nombre_camara
                        metadatos["directores"]["camara"]["cargo"] = cargo_camara
                else:
                    print("No se pudo encontrar el final de la sección de directores.")
            else:
                print("No se encontró la palabra 'DIRECTORES:'")


            # --- 7. Tipo de Documento / Descripción (Regex + spaCy) ---
            #pattern_tipo_documento = r"(?i)(CAMARA DE REPRESENTANTES|PROYECTOS DE LEY|SENADO DE LA REPÚBLICA|COMENTARIOS|PONENCIAS|ACTA|PROYECTO DE LEY|INFORME|RESOLUCIÓN|CONCEPTO|PROPOSICIÓN|CONSTANCIA|OBJECIONES|CONCEPTOS JURÍDICOS|LEYES SANCIONADAS|PRESENTACIÓN)(.*)"
            pattern_tipo_documento = r"(?i)(COMENTARIOS)(.*?)(AL PROYECTO DE LEY NÚMERO \d+ DE \d+ SENADO)?(\".*?\")"
            doc_spacy = nlp(texto_completo)
            for oracion in doc_spacy.sents:
                match = re.search(pattern_tipo_documento, oracion.text, re.MULTILINE | re.DOTALL)
                if match:
                    metadatos["tipo_documento"] = match.group(1).strip()
                    metadatos["subtitulo"] = match.group(2).strip()
                    metadatos["subtitulo_1"] = match.group(4).strip() if match.group(4) else match.group(2).strip()
                    #metadatos["subtitulo_2"] = match.group(6).strip() if match.group(6) else match.group(4).strip()
                    #metadatos["subtitulo_3"] = match.group(13).strip() if match.group(13) else None
                    break   
        
                

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo PDF: {ruta_pdf}")
        return None
    except Exception as e:
        print(f"Error al procesar el PDF {ruta_pdf}: {e}")
        return None

    return metadatos

In [5]:
def extraer_con_regex(patron, texto):
    coincidencia = re.search(patron, texto, re.IGNORECASE)
    if coincidencia:
        return coincidencia.group(1).replace(" ", "") if len(coincidencia.groups()) == 1 else tuple(grupo.replace(" ", "") for grupo in coincidencia.groups())
    return None

In [6]:
def preprocesar_texto(texto):

    texto = texto.replace("-\n", "")
    texto = texto.replace("\n", " ")
    texto = re.sub(r"\s+", " ", texto)
    texto = texto.strip()
    return texto


In [7]:

def romano_a_entero(romano):
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
               'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero


In [8]:
# --- Ejemplo de Uso ---
ruta_gaceta =  r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64_removed.pdf" 
datos_extraidos = extraer_metadatos_gaceta(ruta_gaceta)

if datos_extraidos:
    print(datos_extraidos)

{'nombre': 'Gaceta del Congreso', 'issn': '0123-9066', 'fecha': '28/03/2016', 'entidades': ['SENADO Y CAMARA', 'SENADO', 'CÁMARA'], 'anio_romano': 'XXV', 'anio': 25, 'numero_publicacion': 110, 'rama': 'Rama Legislativa del Poder Público', 'directores': {'senado': {}, 'camara': {}}, 'tipo_documento': 'COMENTARIOS', 'subtitulo': 'COMENTARIOS AL PROYECTO DE LEY NUMERO 97 DE 2015 SENADO', 'subtitulo_1': '"por la cual se prohibe la produccion, comercializacion, exportacion, importacion y distribucion de cualquier variedad de asbesto en Colombia - COMENTARIOS DE LA ASOCIACION COLOMBIANA DE FIBRAS -ASCOLFIBRAS RESENA respecto a la diferencia entre el crisotilo y los asbestos anfiboles CRISOTILO Agosto 2009 "'}
